In [1]:
import os
from openslide import OpenSlide
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd

In [2]:
svs = '/home/kuki2070s2/PycharmProjects/TeamKyu/Michael/5619_Wirtz.Denis_OTS-19_5619-009.svs'
# svss = [_ for _ in os.listdir(src) if _.endswith('svs')]
# svs = svss[0]
# svs = src

# svs = OpenSlide(os.path.join(src,svs))

# xmls = [_ for _ in os.listdir(src) if _.endswith('xml')]
xml = '/home/kuki2070s2/PycharmProjects/TeamKyu/Michael/5619_Wirtz.Denis_OTS-19_5619-009.xml'

tree = ET.parse(xml)
root = tree.getroot()
# pre-allocate arrays of interest
x = np.array([])
y = np.array([])
obj = np.array([])
label = np.array([])

for Annotation in root.iter('Annotation'):
  for Region in Annotation.iter('Region'):
     xx = np.array([int(Vertex.get('X')) for Vertex in Region.iter('Vertex')])
     yy = np.array([int(Vertex.get('Y')) for Vertex in Region.iter('Vertex')])
     objj = np.array([int(Region.get('Id'))]*len(xx))
     labell = np.array([int(Annotation.get('Id'))]*len(xx))
     x = np.concatenate((x,xx),axis=None)
     y = np.concatenate((y,yy),axis=None)
     obj = np.concatenate((obj,objj),axis=None)
     label = np.concatenate((label,labell),axis=None)
mdict = {'x':x,'y':y,'objID':obj,'label':label}
df = pd.DataFrame(mdict)

In [3]:
import openslide
import math

for classidx, classid in enumerate(np.unique(df['label'])):
    for objidx, objid in enumerate(np.unique(df['objID'])):
        try:
            object = df[df['label']==classid][df['objID']==objid]
            x = object['x']
            y = object['y']
            xmin = np.min(x)
            xmax = np.max(x)
            ymin = np.min(y)
            ymax = np.max(y)
            size = (int(xmax-xmin),int(ymax-ymin))
            slide = openslide.OpenSlide(svs)
            region = np.array(slide.read_region(location=(int(xmin),int(ymin)),level=0,size=size))
            vert = np.array([[i-xmin,j-ymin] for i,j in zip(x,y)]).astype(np.int32)
            mask = np.zeros((size[1],size[0]), dtype=np.uint8)
            cv2.fillPoly(mask,[vert],color=255)
            cv2.imwrite('class'+str(classidx)+'obj'+str(objidx)+'.tif', mask)
            cv2.imwrite('class'+str(classidx)+'obj'+str(objidx)+'.jpg', region)
            if np.min(mask.size) < 256:
                x_centroid = np.sum(x)/len(x)
                y_centroid = np.sum(y)/len(y)
                l,r = x_centroid - 128, x_centroid + 128
                u,d = y_centroid - 128, y_centroid + 128
                mask = np.zeros(((d - u), (r - l)), dtype=np.uint8)
                # multiplier = 256 / np.min(mask.size)
                # mask.size = [int(round(multiplier*x)) for x in mask.size]
                # region.size = [int(round(multiplier*x)) for x in region.size]
            else:
                tile_size = (256, 256)
                offset = (256, 256)
                img = mask
                img_shape = mask.shape
                for i in range(int(math.floor(img_shape[0]/(offset[1] * 1.0)))):
                    for j in range(int(math.floor(img_shape[1]/(offset[0] * 1.0)))):
                        cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
                        cv2.imwrite("class" + str(classid) + "object" + str(objid) + "_" + str(i) + "_" + str(j) + "mask" + ".png", cropped_img)
                img = region
                img_shape = region.shape
                for i in range(int(math.floor(img_shape[0]/(offset[1] * 1.0)))):
                    for j in range(int(math.floor(img_shape[1]/(offset[0] * 1.0)))):
                        cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
                        cv2.imwrite("class" + str(classid) + "object" + str(objid) + "_" + str(i) + "_" + str(j) + "image" + ".png", cropped_img)
        except ValueError:
            continue

/home/kuki2070s2/anaconda3/envs/TF2Shared/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [ ]:
#check for images smaller than 256 and increase the size to 256